In [4]:
import os

import pandas as pd
import numpy as np
import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt

import src.visualization as vs
import src.json_cleaning as jc


In [5]:
data = pd.read_csv('data/raw/kickstarter.csv')

def drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """Drop duplicate rows, and rows that are duplicated and rows that only differ in usd_type"""

    df = df.query('~(id.duplicated() & usd_type == "domestic")')
    df = df.drop_duplicates('id', keep='first')
    return df

data = drop_duplicates(data)

In [3]:
# convert goal with static_usd_rate, drop both afterwards and keep converted goal
def goal_convert(df):
    df['converted_goal'] = (df['goal'].mul(df['static_usd_rate'])).round(2)
    df['usd_pledged'] = df['usd_pledged'].round(2)
    df.drop(['goal','static_usd_rate'], axis =1, inplace = True)
    return df

data = goal_convert(data)

data.shape

NameError: name 'data' is not defined

In [ ]:
# 'blurb' and 'name' to len()
def string_length(df):
    df['len_blurb'] = df['blurb'].str.split().str.len()
    df['len_name'] = df['name'].str.split().str.len()
    df.drop(['blurb', 'name'], axis = 1, inplace = True)
    return df

data = string_length(data)

data.shape

In [ ]:
# created and launch time windows
date_columns = ['created_at','deadline','launched_at']

# for column in date_columns:
#     data[column] = data[column].apply(dt.datetime.fromtimestamp)

launch_to_deadline = (data['deadline'] - data['launched_at'])
creation_to_launch = (data['launched_at'] - data['created_at'])

data['launch_to_deadline'] = launch_to_deadline
data['creation_to_launch'] = creation_to_launch

data['month'] = pd.to_datetime(data['launched_at'], unit='s').dt.month_name()
data['weekday'] = pd.to_datetime(data['launched_at'], unit='s').dt.day_name()
data['day_hour'] = pd.to_datetime(data['launched_at'], unit='s').dt.hour

data.drop(date_columns, axis=1, inplace=True)

# launch_to_deadline_days = launch_to_deadline.dt.days
# launch_to_deadline_minutes = launch_to_deadline.dt.seconds // 60

# creation_to_launch_days = creation_to_launch.dt.days
# creation_to_launch_minutes = creation_to_launch.dt.seconds // 60

# time_df = pd.concat([launch_to_deadline_days, launch_to_deadline_minutes,
#                      creation_to_launch_days, creation_to_launch_minutes],
#                     axis=1,
#                     keys = ['launch_to_deadline_days', 'launch_to_deadline_minutes',
#                             'creation_to_launch_days', 'creation_to_launch_minutes'])

In [ ]:
# convert category vars to categories
col = ['category']
data = jc.json_cleaning(data, col)
#data.drop(['category_#slug','category_id'], axis=1, inplace=True)

data['parent_category'] = data['category_slug'].apply(lambda x: x.split('/')[0])
data.head()

In [ ]:
# drop everything that isn't successful or failed
data = data.query('state == "successful" | state == "failed" ')
data.state = data.state.apply(lambda x: 0 if 'failed' in x else 1)

In [ ]:
data = data.query('~(id.duplicated() & usd_type == "domestic")')
data = data.drop_duplicates('id', keep='first')
data

In [ ]:
def drop_duplicates(dataframe):
    # Identify the duplicated rows based on the "id" column
    duplicate_mask = dataframe.duplicated('id').copy()

    # Select the rows that are not duplicates and do not have "domestic" as the value in the "usd_type" column
    dataframe = dataframe[~((duplicate_mask) & (dataframe['usd_type'] == 'domestic'))]
    
    # Drop the rows that are duplicates based on the "id" column, keeping only the first occurrence of each group
    dataframe.drop_duplicates('id', keep='first', inplace=True)

    # Return the resulting DataFrame
    return dataframe

data = drop_duplicates(data)

In [ ]:
data

In [ ]:
drop_these = [
    'converted_pledged_amount',
    'currency',
    'currency_symbol',
    'currency_trailing_code',
    'current_currency',
    'disable_communication',
    'friends',
    'fx_rate',
    'id',
    'is_backing',
    'is_starrable',
    'is_starred',
    'permissions',
    'pledged',
    'slug',
    'source_url',
    'spotlight',
    'state_changed_at',
    'urls',
    'category_slug'
    ]

drop_these_too = [
    'creator',
    'location',
    'photo',
    'profile',
]

data.drop(drop_these, axis=1, inplace=True)
data.drop(drop_these_too, axis=1, inplace=True)

In [ ]:
vs.nice_summary(data)

# sns.histplot(data=time_df, x='launch_to_deadline')

# it looks like projects can be created but not immediately launched
# once a project is launched there is a maximum funding time of 93 days (3 months)

# when do people launch their projects?
# when are they more likely to postpone the launch?

In [ ]:
data.to_csv(os.path.join('data/processed/', 'kickstarter_clean.csv'), index=False)

In [ ]:
sns.countplot(data=data, x='state', hue='usd_type')

In [ ]:
plt.figure(figsize=(20, 30))
plt.subplots_adjust(hspace=0.5)
i = 1;
for name in time_df.columns.tolist():
    plt.subplot(5,2,i)
    sns.boxplot(data=time_df, x=name)
    i = i + 1

In [ ]:
# KEEP ONLY SUCCESSFUL AND FAILED (otherwise comparing apples and pears (:  )

display(data.query('pledged >= goal and state == False').groupby('state').state.count())
display(data.query('pledged <= goal and spotlight == True').groupby('state').state.count())
display(data.groupby('state').state.count())
display(data.groupby('state').state.count())

In [ ]:
sns.countplot(data=data, x='state', hue='spotlight')

In [ ]:
data.query('state=="live"')['state'].count()